## Example 1: Fibonacci Sequence

The fibonacci sequence, when written iteratively as follows, contains a dependency. Here is the relevant code:

```
int fib(int x) {
    int prevPrev = 0;
    int prev = 0;
    int curr = 1;

    # pragma acc kernels
    for (int i=0; i<x; i++) {
        prevPrev = prev;
        prev = curr;
        curr = prevPrev + prev;
    }
    return curr;
}

```

Let's see what the OpenACC Compiler says about this code:

In [3]:
!pgcc -acc -Minfo=accel data_dep_daniel.c



fib:
     10, Loop carried scalar dependence for curr at line 12
         Loop carried scalar dependence for prev at line 11
         Scalar last value needed after loop for curr at line 15
         Accelerator serial kernel generated
         Generating Tesla code
         10, #pragma acc loop seq
     10, Loop carried scalar dependence for curr at line 12
         Loop carried scalar dependence for prev at line 11


## Example 2: 



```
#include <openacc.h>

int main(){
    int N = 20;
    int * b = (int*) malloc(sizeof(int) * N);
    b[1] = 5;
    int * a = (int*) malloc(sizeof(int) * N);
    
    for(int i = 0; i < N; ++i) {
          
    }
}
```


In [4]:
!pgcc -acc -Minfo=accel ex.c

PGC-W-0155-Pointer value created from a nonlong integral type  (ex.c: 5)
PGC-W-0155-Pointer value created from a nonlong integral type  (ex.c: 7)
PGC/x86-64 Linux 19.4-0: compilation completed with warnings


# Example 3:


```
int main(){
    int N = 20;
    int * I = (int*) malloc(sizeof(int) * N);
    int ** a = (int**) malloc(sizeof(int*) * N);
    for(int i = 0; i < N; ++i)
        a[i] = (int*) malloc(sizeof(int) * N);
    
    a[0][0] = 0;
    
    for(int i = 0; i < N; ++i) {
        for(int j = 0; j < N; ++j) {
           a[i][j] = (i != 0) ? ((j != 0) ? a[i-1][j-1] : a[i-1][j]) : a[i][j-1]; 
           if(i == j) I[i] = j;
        }
    }
}
```

In [5]:
!pgcc -acc -Minfo=accel ex2.c

PGC-W-0155-Pointer value created from a nonlong integral type  (ex2.c: 3)
PGC-W-0155-Pointer value created from a nonlong integral type  (ex2.c: 4)
PGC-W-0155-Pointer value created from a nonlong integral type  (ex2.c: 6)
PGC/x86-64 Linux 19.4-0: compilation completed with warnings


## Example 4: Flow Dependency


Kobe Davis
Prof. Karavanic
CS 405
1 June 2019

This is an example of a matrix operation which contains a flow dependency. Despite the flow dependency, the code may be refactored to execute in parallel. The parallel version of this program, found in the file (flowdep_refactor_par.cpp) demonstrates a solution through refactoring.


```
#include <cstdlib>
#include <ctime>

const int DIM = 30; // Height and Width

int main() {
    srand(time(NULL));

    // Declare matrix of height and width, DIM
    int data[DIM][DIM];

    // Randomly initialize elements in range (0, 1)
    for(int i = 0; i < DIM; ++i)
        for(int j = 0; j < DIM; ++j)
            data[i][j] = rand() % 2;

    // If the elements directly above and directly to
    // the left are equal, set the current element to match them
    // Otherwise do nothing
    #pragma acc parallel loop
    for(int i = 1; i < DIM; ++i)
        for(int j = 1; j < DIM; ++j)
            if(!data[i-1][j] == !data[i][j-1]) // XOR
                data[i][j] = data[i-1][j] && data[i][j-1];

    // There is a flow depency in the above code because our calculation of
    // data[i][j] relies on both the previous row and previous column. A partial
    // parallelization will not work because a dependency in both the previous
    // row and column stops us from simply parallelizing a single loop.

    return 0;
}
```

In [6]:
!pgc++ -acc -Minfo=accel flowdep_refactor_seq.cpp

main:
     28, Generating Tesla code
         34, #pragma acc loop gang, vector(29) /* blockIdx.x threadIdx.x */
         35, #pragma acc loop seq
     28, Generating implicit copy(data[:][:])
     35, Loop carried dependence of data prevents parallelization
         Loop carried backward dependence of data prevents vectorization


## Solution to Example 4


```

\
```

In [ ]:
!pgc++ -acc -Minfo=accel flowdep_refactor_par.cpp